In [ ]:
# Importing the requisite packages
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pysal
import splot

In [ ]:
### Setting the file path
data_path = os.path.join(".", "data")

In [ ]:
# Reading in the data - REMINDER: You'll need to download this data yourself from InsideAirbnb
listings = pd.read_csv(os.path.join(data_path, "NY Airbnb April 2024", "listings.csv.gz"), compression="gzip")

## Exploratory Data Analysis

In [ ]:
print(listings.columns)

In [ ]:
# Subsetting the data
listings_sub = listings[['id','property_type','neighbourhood_cleansed', 'neighbourhood_group_cleansed','beds','bathrooms', 'price','latitude','longitude']]

listings_sub.head()

In [ ]:
listings_sub.info()

In [ ]:
# Cleaning the data
listings_sub.drop(columns=['beds','bathrooms'], inplace=True)
listings_sub["price"] = listings_sub["price"].replace("[$,]", "", regex=True).astype(float)

In [ ]:
listings_sub

In [ ]:
listings_sub['price'].describe()

In [ ]:
%matplotlib inline
plt.hist(listings_sub['price'], bins=400, density=True, alpha=0.6)
plt.xscale('log')
plt.show()

In [ ]:
import seaborn as sns
sns.distplot(listings_sub['price'], kde=True)
plt.show()

## Exploratory Spatial Data Analysis

In [ ]:
# Convert the pandas dataframe to a geopandas dataframe
listings_sub_gpd = gpd.GeoDataFrame(listings_sub, geometry=gpd.points_from_xy(listings_sub.longitude, listings_sub.latitude, crs=4326))

In [ ]:
import geoplot.crs as gcrs
import geoplot as gplt
ax = gplt.webmap(listings_sub_gpd,projection=gcrs.WebMercator())
gplt.pointplot(listings_sub_gpd, ax=ax)

In [ ]:
# Borough Boundaries
boroughs = gpd.read_file(gplt.datasets.get_path("nyc_boroughs"))
boroughs.plot()

In [ ]:
ax = gplt.polyplot(boroughs, projection=gcrs.WebMercator(), zorder=1)
gplt.kdeplot(listings_sub_gpd, ax=ax, cmap="Reds")

In [ ]:
# Reading in the New York Census Tracts
NY_tracts_path = "https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_36_tract.zip"
NY_Tracts = gpd.read_file(NY_tracts_path)
NY_Tracts = NY_Tracts.to_crs(4326)

# Subsetting the census tracts to those in the New York CBSA
cbsa_path = 'https://www2.census.gov/geo/tiger/TIGER2021/CBSA/tl_2021_us_cbsa.zip'
cbsas = gpd.read_file(cbsa_path)
NY_cbsa = cbsas[cbsas['GEOID']=='35620']

mask = NY_Tracts.intersects(NY_cbsa.loc[620,'geometry'])
NY_Tracts_subset = NY_Tracts.loc[mask]

# Aggregating the airbnb locations to the NY census tracts
NY_Tracts_sj = gpd.sjoin(NY_Tracts_subset, listings_sub_gpd, how='left', predicate='contains')
NY_Tracts_sj = NY_Tracts_sj[['GEOID','price','geometry']]
NY_Tracts_Agg = NY_Tracts_sj.dissolve(by='GEOID', aggfunc='mean')

NY_Tracts_Agg

In [ ]:
gplt.choropleth(NY_Tracts_Agg.dropna(), hue="price", cmap="Greens", figsize=(60,30), legend=True)

In [ ]:
import geoviews
geoviews.extension("bokeh")

choropleth = geoviews.Polygons(data=NY_Tracts_Agg, vdims=["price","GEOID"])

choropleth.opts(height=600, width=900, title="NYC Airbnb Price",
                tools=["hover"], cmap="Greens", colorbar=True, colorbar_position="bottom")

In [ ]:
import statistics

# Calculating the mean and standard deviation
mean_price = statistics.mean(NY_Tracts_Agg['price'].dropna())
stdev = statistics.stdev(NY_Tracts_Agg['price'].dropna())

print("The mean Airbnb price is: % s " % (round(mean_price, 2)))
print("The standard deviation of Airbnb prices is: % s " % (round(stdev, 2)))

# Droping records that are more than 1 standard deviation from the mean
NY_Tracts_Agg_filtered = NY_Tracts_Agg[NY_Tracts_Agg['price'] < mean_price+stdev]

choropleth = geoviews.Polygons(data=NY_Tracts_Agg_filtered, vdims=["price","GEOID"])

choropleth.opts(height=600, width=900, title="NYC Airbnb Price",
                tools=["hover"], cmap="Greens", colorbar=True, colorbar_position="bottom")